# IBM Quantum Challenge 2020 November - Exercise 2-a

[Notebook containing the challenge exercise](https://github.com/qiskit-community/IBMQuantumChallenge2020/blob/main/exercises/week-2/ex_2a_en.ipynb)

## Dependencies

In [1]:
from qiskit import Aer, ClassicalRegister, execute, QuantumCircuit, QuantumRegister
import numpy as np

## The Mathematical Solution

In [2]:
def comp_clauses(qc, q_s, q_l):
    """Function to implement the mathematical solution.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    q_s : list
        Superposed qbits.
    q_l : list
        Light state qbits.
    """

    # the neighbour matrix
    A = [   [1, 1, 0, 1, 0, 0, 0, 0, 0],
            [1, 1, 1, 0, 1, 0, 0, 0, 0],
            [0, 1, 1, 0, 0, 1, 0, 0, 0],
            [1, 0, 0, 1, 1, 0, 1, 0, 0],
            [0, 1, 0, 1, 1, 1, 0, 1, 0],
            [0, 0, 1, 0, 1, 1, 0, 0, 1],
            [0, 0, 0, 1, 0, 0, 1, 1, 0],
            [0, 0, 0, 0, 1, 0, 1, 1, 1],
            [0, 0, 0, 0, 0, 1, 0, 1, 1] ]

    # for each element in A
    for i in range(9):
        for j in range(9):
            # apply CNOTs whereever an element is 1
            if A[i][j] == 1:
                qc.cx(q_s[i], q_l[j])

## The Oracle

In [3]:
def comp_oracle(qc, q_s, q_l, q_a, q_f):
    """Function to implement the oracle.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    q_s : list
        Superposed qbits.
    q_l : list
        Light state qbits.
    q_a : list
        Ancilla qbits.
    q_f : int
        Flag qbit.
    """

    # compute clauses
    comp_clauses(qc, q_s, q_l)

    # flag the solutions
    qc.mct(q_l, q_f, ancilla_qubits=q_a, mode='v-chain')

    # uncompute clauses
    comp_clauses(qc, q_s, q_l)

## The Diffuser

In [4]:
def get_diffuser_gate(num):   
    """Function to implement a general diffuser gate.

    Parameters
    ----------
    num : int
        Number of superposed states.

    Returns
    -------
    D : :class:`qiskit.circuit.Gate`
        Quantum gate representing the circuit.
    """

    # num-bit register for the superposed qbits
    q_s = QuantumRegister(num)
    # (num - 2)-bit register for the ancilla qbits
    q_a = QuantumRegister(num - 3)
    # quantum circuit
    qc = QuantumCircuit(q_s, q_a)

    # apply H and X gates to all qbits
    for i in range(num):
        qc.h(i)
        qc.x(i)

    # implement controlled-Z gate
    # apply H gate to the final qbit
    qc.h(num - 1)
    # flip the final bit
    qc.mct(list(range(num - 1)), num - 1, ancilla_qubits=q_a, mode='v-chain')
    # apply H gate to the final qbit
    qc.h(num - 1)

    # apply X and H gates to all qbits
    for i in range(num):
        qc.x(i)
        qc.h(i)

    # return circuit as gate
    return qc.to_gate()

## The Answer Function

In [5]:
def week2a_ans_func(lights):
    """Function to return a quantum circuit to obtain the solution for a given sequence of light states.

    Parameters
    ----------
    lights : list
        Sequence of light states.

    Returns
    -------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum circuit to obtain the solution.
    """

    # register for superposed qbits
    q_s = QuantumRegister(9, 'q_s')
    # register for light state qbits
    q_l = QuantumRegister(9, 'q_l')
    # register for ancilla qbits
    q_a = QuantumRegister(7, 'q_a')
    # register for flag qbit
    q_f = QuantumRegister(1, 'q_f')
    # classical register to measure solution
    c = ClassicalRegister(9, 'c')
    # quantum circuit
    qc = QuantumCircuit(q_s, q_l, q_a, q_f, c)

    # initialize superposed qbits
    for i in range(9):
        qc.h(q_s[i])

    # initialize light states complementary to given states
    # such that the clauses return an odd sum for solution qbits
    for i in range(9):
        if lights[i] == 0:
            qc.x(q_l[i])

    # initialize the flag qbit
    qc.x(q_f[0])
    qc.h(q_f[0])

    # oracle-diffuser loop
    for i in range(1):
        # compute oracle
        comp_oracle(qc, q_s, q_l, q_a, q_f)
        # compute diffuser for light states
        qc.append(get_diffuser_gate(9), list(range(9)) + list(range(18, 24)))

    # measure
    qc.measure(q_s, c)

    # return reversed bits for same endian
    return qc.reverse_bits()

## The Quantum Circuit

In [6]:
# sample sequences and solutions
Q = [   [[0, 1, 1, 1, 0, 0, 1, 1, 1], '110010101'],
        [[1, 1, 0, 0, 1, 0, 0, 0, 0], '111100011'],
        [[1, 1, 1, 1, 0, 0, 0, 1, 0], '110110011'],
        [[0, 0, 1, 1, 1, 0, 0, 0, 1], '101100101'],
        [[1, 1, 1, 1, 1, 1, 1, 1, 0], '110110000'],
        [[0, 0, 0, 1, 0, 1, 1, 0, 1], '000000101']  ]

# selected sequence of light states
lights = Q[0][0]

# get the quantum circuit
qc = week2a_ans_func(lights)

# # display
# qc.draw(output='mpl')

## Simulation

In [7]:
# execute the quantum circuit on the backend
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=8000)

# obtain the maximum from the result
result = job.result()
counts = result.get_counts()
max_key = max(counts, key=counts.get)
print('{}: {}'.format(max_key, counts[max_key]))

110010101: 133


## Grading

In [8]:
# dependencies for calculating cost
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# calculate cost
ur = Unroller(['u3', 'cx'])
pm = PassManager(ur)
qc_cost = pm.run(qc) 
op_dict = qc_cost.count_ops()
# given criteria to calculate cost
print('Cost: {}'.format(op_dict['u3'] + op_dict['cx'] * 10))

Cost: 1786
